Zadanie 1

In [14]:
import numpy as np

# 1.1 Wczytanie danych
crypto = pyreadr.read_r('crypto.rds')
df = crypto[None] # DataFrame z R
df.head()

# 1.2 Dane dotyczące Bitcoina
df = df[df['Currency'].str.contains('bitcoin', case=False, na=False)]

# 1.3 Pozostawienie kolumn 'Date' i 'Close'
df = df[['Date', 'Close']]
df.head()

,Date,Close
6162,"Feb 11, 2018",1217.64
6163,"Feb 10, 2018",1257.31
6164,"Feb 09, 2018",1312.11
6165,"Feb 08, 2018",1284.56
6166,"Feb 07, 2018",957.96
